## Build a vector store from a duckdb table using custom code

In [1]:
import os

# Please change the paths to your own paths
paper_parser_dir = "/Users/jy006/Documents/Code/BioMedGPS/paper-parser"
parquet_file = "/Users/jy006/Documents/Code/BioMedGPS/paper-parser/benchmarks/antibody/text_chunks/paragraphs.parquet"

import os
import sys

script_dir = os.path.join(paper_parser_dir, "scripts")
sys.path.append(script_dir)

print(sys.path)

['/Users/jy006/miniconda3/envs/paper-parser/lib/python310.zip', '/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10', '/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10/lib-dynload', '', '/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10/site-packages', '/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10/site-packages/setuptools/_vendor', '/Users/jy006/Documents/Code/BioMedGPS/paper-parser/scripts']


In [ ]:
!pip install -qU qdrant-client duckdb

In [2]:
import duckdb

conn = duckdb.connect()
conn.sql(f"SELECT COUNT(*) FROM read_parquet('{parquet_file}')").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          961 │
└──────────────┘



In [3]:
from embedding import DuckDBLoader

raw_documents = DuckDBLoader(
    query=f"SELECT * FROM read_parquet('{parquet_file}') LIMIT 100",
    page_content_column="text",
    metadata_columns=["pmid", "pmc", "doi", "pubdate"],
).load()

sentences = [doc.get("text") for doc in list(raw_documents)[:100]]

__init__ took 0.00 milliseconds
load took 0.00 milliseconds


### Run NV-Embed-v2

In [1]:
from sentence_transformers import SentenceTransformer
import torch

model =  SentenceTransformer("nvidia/NV-Embed-v2", trust_remote_code=True, model_kwargs={"torch_dtype": torch.float16}, device="mps")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
import os
from embedding import build_vector_db

vector_db_dir = "/Users/jy006/Documents/Code/BioMedGPS/paper-parser/benchmarks/antibody/pmc_embedding/test_rocksdb_no_speedup"
batch_size = 8
num_documents = 100

build_vector_db(
    cache_filepath=vector_db_dir,
    raw_document_path=parquet_file,
    # model_name="nvidia/NV-Embed-v2",
    model=model,
    page_content_column="text",
    metadata_columns=["pmid", "pmc", "doi", "pubdate"],
    num_documents=num_documents,
    batch_size=batch_size,
    allow_batch_mode=True
)

Loading documents...
__init__ took 0.00 milliseconds
load took 0.00 milliseconds
Initializing vector database...
Loading model...
Initializing cache database...
__init__ took 68.21 milliseconds
Computing embeddings and inserting into vector database...
Starting multi-process pool...
Processing batch 1
Time taken to check: 0.0017609596252441406
Missing 0 texts, 8 cached
Processing batch 2
Time taken to check: 0.0004668235778808594
Missing 0 texts, 8 cached
Processing batch 3
Time taken to check: 0.0005509853363037109
Missing 0 texts, 8 cached
Processing batch 4
Time taken to check: 6.818771362304688e-05
Missing 8 texts, 0 cached


/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
/Users/jy006/miniconda3/envs/paper-parser/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
/Users/jy006/miniconda

Time taken to encode: 481.90996074676514
Time taken to put: 0.011041879653930664
Time taken to release GPU memory: 0.00011491775512695312
Processing batch 5
Time taken to check: 6.508827209472656e-05
Missing 8 texts, 0 cached


KeyboardInterrupt: 

### Run all-MiniLM-L6-v2

In [3]:
import os
from embedding import build_vector_db

vector_db_dir = "/Users/jy006/Documents/Code/BioMedGPS/paper-parser/benchmarks/antibody/pmc_embedding/test_rocksdb_no_speedup_1"
batch_size = 8
num_documents = 0

build_vector_db(
    cache_filepath=vector_db_dir,
    raw_document_path=parquet_file,
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    # model=model,
    page_content_column="text",
    metadata_columns=["pmid", "pmc", "doi", "pubdate"],
    num_documents=num_documents,
    batch_size=batch_size,
    allow_batch_mode=True
)

2025-01-25 09:53:37,695 - INFO - Loading documents...
INFO:embedding:Loading documents...
2025-01-25 09:53:37,696 - INFO - __init__ took 0.00 milliseconds
INFO:embedding:__init__ took 0.00 milliseconds
2025-01-25 09:53:37,696 - INFO - load took 0.00 milliseconds
INFO:embedding:load took 0.00 milliseconds
2025-01-25 09:53:37,697 - INFO - Initializing vector database...
INFO:embedding:Initializing vector database...
2025-01-25 09:53:37,698 - INFO - Loading model...
INFO:embedding:Loading model...
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-01-25 09:53:38,537 - INFO - Initializing cache database...
INFO:embedding:Initializing cache database...
2025-01-25 09:53:38,592 - INFO - __init__ took 894.19 milliseconds
INFO:embedding:__init__ took 894.19 milliseconds
2025-01-25 09:53:38,593 - INFO - Computing embeddings and inserting into ve

Chunks:   0%|          | 0/8 [00:00<?, ?it/s]

2025-01-25 09:53:47,275 - INFO - Time taken to encode: 0.10180878639221191
INFO:embedding:Time taken to encode: 0.10180878639221191
2025-01-25 09:53:47,277 - INFO - Time taken to put: 0.0017070770263671875
INFO:embedding:Time taken to put: 0.0017070770263671875
2025-01-25 09:53:47,278 - INFO - Time taken to release GPU memory: 2.9087066650390625e-05
INFO:embedding:Time taken to release GPU memory: 2.9087066650390625e-05
2025-01-25 09:53:47,282 - INFO - 
Processing batch 2
INFO:embedding:
Processing batch 2
2025-01-25 09:53:47,283 - INFO - Time taken to check: 0.00023174285888671875
INFO:embedding:Time taken to check: 0.00023174285888671875
2025-01-25 09:53:47,283 - INFO - Missing 8 texts, 0 cached
INFO:embedding:Missing 8 texts, 0 cached


Chunks:   0%|          | 0/8 [00:00<?, ?it/s]

2025-01-25 09:53:47,377 - INFO - Time taken to encode: 0.09264302253723145
INFO:embedding:Time taken to encode: 0.09264302253723145
2025-01-25 09:53:47,380 - INFO - Time taken to put: 0.002841949462890625
INFO:embedding:Time taken to put: 0.002841949462890625
2025-01-25 09:53:47,381 - INFO - Time taken to release GPU memory: 1.7881393432617188e-05
INFO:embedding:Time taken to release GPU memory: 1.7881393432617188e-05
2025-01-25 09:53:47,384 - INFO - 
Processing batch 3
INFO:embedding:
Processing batch 3
2025-01-25 09:53:47,385 - INFO - Time taken to check: 0.0002639293670654297
INFO:embedding:Time taken to check: 0.0002639293670654297
2025-01-25 09:53:47,386 - INFO - Missing 8 texts, 0 cached
INFO:embedding:Missing 8 texts, 0 cached


...

2025-01-25 09:54:01,302 - INFO - Time taken to encode: 0.14697003364562988
INFO:embedding:Time taken to encode: 0.14697003364562988
2025-01-25 09:54:01,304 - INFO - Time taken to put: 0.0015518665313720703
INFO:embedding:Time taken to put: 0.0015518665313720703
2025-01-25 09:54:01,305 - INFO - Time taken to release GPU memory: 1.7881393432617188e-05
INFO:embedding:Time taken to release GPU memory: 1.7881393432617188e-05
2025-01-25 09:54:01,307 - INFO - 
Processing batch 120
INFO:embedding:
Processing batch 120
2025-01-25 09:54:01,308 - INFO - Time taken to check: 0.0002853870391845703
INFO:embedding:Time taken to check: 0.0002853870391845703
2025-01-25 09:54:01,309 - INFO - Missing 8 texts, 0 cached
INFO:embedding:Missing 8 texts, 0 cached


Chunks:   0%|          | 0/8 [00:00<?, ?it/s]

2025-01-25 09:54:01,405 - INFO - Time taken to encode: 0.0954430103302002
INFO:embedding:Time taken to encode: 0.0954430103302002
2025-01-25 09:54:01,408 - INFO - Time taken to put: 0.002161264419555664
INFO:embedding:Time taken to put: 0.002161264419555664
2025-01-25 09:54:01,409 - INFO - Time taken to release GPU memory: 3.409385681152344e-05
INFO:embedding:Time taken to release GPU memory: 3.409385681152344e-05
2025-01-25 09:54:01,413 - INFO - 
Processing batch 121
INFO:embedding:
Processing batch 121
2025-01-25 09:54:01,413 - INFO - Time taken to check: 0.00010180473327636719
INFO:embedding:Time taken to check: 0.00010180473327636719
2025-01-25 09:54:01,414 - INFO - Missing 1 texts, 0 cached
INFO:embedding:Missing 1 texts, 0 cached


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

2025-01-25 09:54:01,432 - INFO - Time taken to encode: 0.017298221588134766
INFO:embedding:Time taken to encode: 0.017298221588134766
2025-01-25 09:54:01,433 - INFO - Time taken to put: 0.0003840923309326172
INFO:embedding:Time taken to put: 0.0003840923309326172
2025-01-25 09:54:01,434 - INFO - Time taken to release GPU memory: 6.9141387939453125e-06
INFO:embedding:Time taken to release GPU memory: 6.9141387939453125e-06
2025-01-25 09:54:01,454 - INFO - batch_insert_documents_with_pool took 22.86 seconds
INFO:embedding:batch_insert_documents_with_pool took 22.86 seconds
2025-01-25 09:54:01,455 - INFO - Vector store successfully created and persisted.
INFO:embedding:Vector store successfully created and persisted.
2025-01-25 09:54:01,455 - INFO - build_vector_db took 23.76 seconds
INFO:embedding:build_vector_db took 23.76 seconds


## Example: Build a vector store from a duckdb table

### Install dependencies

In [11]:
!pip install -qU langchain-community duckdb chromadb "langchain-chroma>=0.1.2" ipywidgets sentence-transformers einops datasets python-dotenv

### Load the environment variables

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="/Users/jy006/.ssh/.env")

### Test the duckdb loader

In [6]:
from langchain_community.document_loaders import DuckDBLoader

In [ ]:
loader = DuckDBLoader(
    "SELECT * FROM read_parquet('/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/processed/oa_other/paragraphs/0a96d2c04ab604cd71eed3b268c298c9_20250103_135911.parquet')",
    page_content_columns=["text"],
    metadata_columns=["pmid", "pmc", "doi", "pubdate"],
)

data = loader.load()
data[0]

### Build the knowledge base

In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings, OllamaEmbeddings, SentenceTransformerEmbeddings
from langchain_community.document_loaders import DuckDBLoader
from langchain_community.vectorstores import Chroma

import os

def load_vector_store(chroma_db_dir, num_documents=1000):
    try:
        # Load the document
        raw_documents = DuckDBLoader(
            query=f"SELECT * FROM read_parquet('/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/processed/oa_other/paragraphs/0a96d2c04ab604cd71eed3b268c298c9_20250103_135911.parquet') LIMIT {num_documents}",
            page_content_columns=["text"],
            metadata_columns=["pmid", "pmc", "doi", "pubdate"],
        ).load()

        def preprocess_metadata(documents):
            for doc in documents:
                if doc.metadata:
                    # 替换 metadata 中的 None 为 ""
                    doc.metadata = {k: (v if v is not None else "") for k, v in doc.metadata.items()}
            return documents

        # # Split the document into smaller chunks
        # text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        # documents = text_splitter.split_documents(raw_documents)

        # Embed each chunk and load it into the vector store
        vector_store = Chroma.from_documents(
            documents=preprocess_metadata(raw_documents),
            # embedding=OllamaEmbeddings(model="mistral:7b"),
            # embedding=OpenAIEmbeddings(),
            embedding=SentenceTransformerEmbeddings(
                model_name="nvidia/NV-Embed-v2", model_kwargs={"trust_remote_code": True}
            ),
            persist_directory=str(chroma_db_dir),
        )
        vector_store.persist()
        print("Vector store successfully created and persisted.")
    except Exception as e:
        print(f"Error loading vector store: {e}")


# Path to Chroma vector database
# chroma_db_dir = os.path.abspath("./chroma_db_chatgpt")
chroma_db_dir = os.path.abspath("./chroma_db_nvidia")

print("Chroma vector database path:", chroma_db_dir)

# Load the vector store
if not os.path.exists(chroma_db_dir):
    print("Chroma database not found. Creating a new one...")
    load_vector_store(chroma_db_dir, num_documents=100)
else:
    print("Chroma database found. Skipping creation.")

In [ ]:
import os

from phi.agent import Agent
from phi.knowledge.langchain import LangChainKnowledgeBase
from phi.model.ollama.chat import Ollama
from phi.model.openai.chat import OpenAIChat
from langchain_chroma import Chroma
from langchain_community.embeddings import OpenAIEmbeddings, OllamaEmbeddings

# Path to Chroma vector database
chroma_db_dir = os.path.abspath("./chroma_db_chatgpt")
# os.makedirs(chroma_db_dir, exist_ok=True)
print("Chroma vector database path:", chroma_db_dir)

# Get the vector database
db = Chroma(
    # embedding_function=OllamaEmbeddings(model="mistral:7b"),
    embedding_function=OpenAIEmbeddings(),
    persist_directory=str(chroma_db_dir),
)

# Check if Chroma database has any documents
if not db._collection.count():
    print("Chroma database is empty. Please ensure documents are loaded.")
else:
    print(f"Chroma database contains {db._collection.count()} documents.")

In [ ]:
import duckdb

conn = duckdb.connect()
data = conn.sql("SELECT * FROM read_parquet('/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/processed/oa_other/paragraphs/0a96d2c04ab604cd71eed3b268c298c9_20250103_135911.parquet')")

data.show()

In [ ]:
# Count the number of unique pmc
conn.sql("SELECT COUNT(DISTINCT pmc) FROM read_parquet('/Volumes/Backup/ProjectData/Papers/PMC_OA_Bulk/processed/oa_other/paragraphs/0a96d2c04ab604cd71eed3b268c298c9_20250103_135911.parquet')").show()


In [ ]:
# Create a retriever from the vector store
retriever = db.as_retriever()

docs = retriever.get_relevant_documents("observed base pair difference", k=10)
if not docs:
    print("No relevant documents retrieved.")
else:
    print(f"Retrieved {len(docs)} documents.")

    # Create a knowledge base from the vector store
    knowledge_base = LangChainKnowledgeBase(retriever=retriever, num_documents=10)

    # Initialize the Agent
    # https://docs.phidata.com/agents/knowledge#step-3-agentic-rag
    kb_agent = Agent(
        model=OpenAIChat(id="gpt-4o"),
        # model=Ollama(id="mistral:7b"),
        knowledge_base=knowledge_base,
        add_reference_to_prompt=True,
        # add_references=True,
        instructions=[
            "Always prioritize information from the knowledge base over your training data.",
            "If the knowledge base does not contain information relevant to the query, respond with: 'No relevant information found in the knowledge base.'",
            "Do not generate answers based on prior training data unless explicitly instructed.",
        ],
        markdown=True,
        # debug_mode=True,
    )

    # Test the Agent with a query
    kb_agent.print_response(
        "What was the observed base pair difference between many of the strain types?"
    )

## Run deepspeed

In [ ]:
!export TRITON_CACHE_DIR=/tmp && deepspeed --num_gpus 2 --master_port 60000 /work/data/projects/data2report/deepspeed/run_deepspeed.py